In [4]:
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import string
import pickle

# Tải dữ liệu và model đã lưu
df_business = pickle.load(open('D:\\Workspace\\Data_Mining\\Restaurant_Recomendation_System\\models\\recom_business.pkl', 'rb'))
Q = pickle.load(open('D:\\Workspace\\Data_Mining\\Restaurant_Recomendation_System\\models\\recom_q.pkl', 'rb'))
user_id_vectorizer = pickle.load(open('D:\\Workspace\\Data_Mining\\Restaurant_Recomendation_System\\models\\recom_user_id_vec.pkl', 'rb'))

stop_words = list(stopwords.words('english'))

# Hàm xử lý văn bản
def text_process(text):
    """
    Loại bỏ dấu câu và stop words.
    """
    nopunc = [char for char in text if char not in string.punctuation]
    return " ".join([word for word in nopunc if word.lower() not in stop_words])

# Hàm gợi ý
def recommender(input_text):
    # Chuẩn bị dữ liệu đầu vào
    test_df = pd.DataFrame([input_text], columns=['text'])
    test_df['text'] = test_df['text'].apply(text_process)

    # Vector hóa văn bản
    test_vectors = user_id_vectorizer.transform(test_df['text'])
    test_v_df = pd.DataFrame(test_vectors.toarray(), index=test_df.index, columns=user_id_vectorizer.get_feature_names_out())

    # Tính toán rating dự đoán
    predictItemRating = pd.DataFrame(np.dot(test_v_df.loc[0], Q.T), index=Q.index, columns=['Rating'])
    topRecommendations = predictItemRating.sort_values('Rating', ascending=False)[:5]

    # Lấy thông tin gợi ý
    recommendations = []
    for i in topRecommendations.index:
        temp = {
            'name': df_business[df_business['business_id'] == i]['name'].iloc[0],
            'categories': df_business[df_business['business_id'] == i]['categories'].iloc[0],
            'rating': df_business[df_business['business_id'] == i]['stars'].iloc[0],
        }
        recommendations.append(temp)
    return recommendations

# Nhập văn bản từ người dùng
if __name__ == "__main__":
    input_text = input("Nhập Input: ")
    recommendations = recommender(input_text)

    # Hiển thị kết quả
    print("\nGợi ý nhà hàng:")
    for idx, rec in enumerate(recommendations, 1):
        print(f"{idx}. {rec['name']} (Rating: {rec['rating']})")
        print(f"   Categories: {rec['categories']}")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Nhập Input: I want to have breakfast

Gợi ý nhà hàng:
1. Circle K (Rating: 2.5)
   Categories: Automotive, Gas Stations, Food, Convenience Stores
2. Auntie Anne's (Rating: 1.0)
   Categories: Pretzels, Food
3. Walgreens (Rating: 3.0)
   Categories: Shopping, Convenience Stores, Photography Stores & Services, Food, Cosmetics & Beauty Supply, Beauty & Spas, Drugstores
4. Hardee's (Rating: 3.0)
   Categories: Burgers, Breakfast & Brunch, Restaurants, Fast Food
5. Pilot Travel Center (Rating: 3.0)
   Categories: Automotive, Gas Stations, Food, Convenience Stores
